In [1]:
import torch
import datetime
from torch.autograd import Variable
from torchvision.models import resnet
from torchvision.datasets import coco
from torchvision import transforms

In [4]:
BASE = '/Users/ajitpunj/Documents/School/cs 349d/project/'
m = resnet.resnet18(pretrained=True)

In [5]:
ANN_PATH = 'coco/cocoapi/annotations/instances_val2017.json'
VAL_PATH = 'coco/cocoapi/images/val2017/'
test_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                      std=[0.229, 0.224, 0.225])
])
def merge_coco(l):
    return torch.stack([x[0] for x in l])
c_dataset = coco.CocoDetection(root=BASE+VAL_PATH, annFile=BASE+ANN_PATH, transform=test_transform, )
eval_loader = torch.utils.data.DataLoader(c_dataset, batch_size=1, shuffle=True, collate_fn=merge_coco)

loading annotations into memory...
Done (t=0.74s)
creating index...
index created!


In [6]:
def compute_mem_size(t):
    t_size = t.size()
    n_nums = 1
    for n in t_size:
        n_nums *= n
    bit_size_map = {torch.float32 : 32, torch.float16 : 16, torch.float64 : 64, torch.uint8 : 8, torch.int8 : 8, torch.int16 : 16, torch.int32 : 32, torch.int64 : 64}
    return (n_nums * bit_size_map[t.dtype]) / 8

def resnet_measure_forward(model, x):
    print('Image size:', compute_mem_size(x))
    t0 = datetime.datetime.now()
    x = model.conv1(x)
    x = model.bn1(x)
    x = model.relu(x)
    t1 = datetime.datetime.now()
    dt1 = t1 - t0
    print('After conv1 (' + str(dt1.total_seconds()) + '):', compute_mem_size(x))
    t2 = datetime.datetime.now()
    x = model.maxpool(x)
    t3 = datetime.datetime.now()
    dt2 = t3 - t2
    print('After maxpool (' + str(dt2.total_seconds()) + '):', compute_mem_size(x))
    t4 = datetime.datetime.now()
    x = model.layer1(x)
    t5 = datetime.datetime.now()
    dt3 = t5 - t4
    print('After l1 (' + str(dt3.total_seconds()) + '):', compute_mem_size(x))
    t6 = datetime.datetime.now()
    x = model.layer2(x)
    t7 = datetime.datetime.now()
    dt4 = t7 - t6
    print('After l2 (' + str(dt4.total_seconds()) + '):', compute_mem_size(x))
    t8 = datetime.datetime.now()
    x = model.layer3(x)
    t9 = datetime.datetime.now()
    dt5 = t9 - t8
    print('After l3 (' + str(dt5.total_seconds()) + '):', compute_mem_size(x))
    t10 = datetime.datetime.now()
    x = model.layer4(x)
    t11 = datetime.datetime.now()
    dt6 = t11 - t10
    print('After l4 (' + str(dt6.total_seconds()) + '):', compute_mem_size(x))
    t12 = datetime.datetime.now()
    x = model.avgpool(x)
    t13 = datetime.datetime.now()
    dt7 = t13 - t12
    print('After avgpool (' + str(dt7.total_seconds()) + '):', compute_mem_size(x))
    x = x.view(x.size(0), -1)
    t14 = datetime.datetime.now()
    x = model.fc(x)
    t15 = datetime.datetime.now()
    dt8 = t15 - t14
    print('After fc (' + str(dt8.total_seconds()) + '):', compute_mem_size(x))
    return x

In [7]:
m.eval()
for batch in eval_loader:
    resnet_measure_forward(m, batch)
    break

Image size: 602112.0
After conv1 (0.041257): 3211264.0
After maxpool (0.010452): 802816.0
After l1 (0.03902): 802816.0
After l2 (0.026047): 401408.0
After l3 (0.019898): 200704.0
After l4 (0.023072): 100352.0
After avgpool (0.00138): 2048.0
After fc (0.003293): 4000.0


In [ ]:
transforms.ToPILImage()(sample_input)